In [ ]:
def scrapeAdvancedPass(year):
    from splinter import Browser
    from bs4 import BeautifulSoup as soup
    from webdriver_manager.chrome import ChromeDriverManager
    import pandas as pd
    from functools import reduce
    # Set up Splinter
    executable_path = {'executable_path': ChromeDriverManager().install()}
    browser = Browser('chrome', **executable_path, headless=True)
    # Visit the AdvancedStats to Scrape site
    url = f'https://www.pro-football-reference.com/years/{year}/passing_advanced.htm'
    browser.visit(url)
    # Parse the HTML
    html = browser.html
    html_soup = soup(html, 'html.parser')
    try:
        
        df1 = html_soup.find('table', {"id" : 'advanced_air_yards'})
        df2 = html_soup.find('table', {"id" : 'advanced_accuracy'})
        df3 = html_soup.find('table', {"id" : 'advanced_pressure'})
        df4 = html_soup.find('table', {"id" : 'advanced_play_type'})

        df1 = pd.read_html(str(df1),header=1)[0]
        df2 = pd.read_html(str(df2),header=1)[0]
        df3 = pd.read_html(str(df3),header=1)[0]
        df4 = pd.read_html(str(df4),header=1)[0]
    
        data_frames = [df1, df2, df3, df4]
        df = reduce(lambda  left,right: pd.merge(left,right,on=['Player'],
                                                    how='outer'), data_frames)
        df = df[df['Player'] != 'Player']
        df = df[df['Player'].notna()]
        df = df.replace('\/','',regex=True).astype(object)
        df = df.replace('^qb','QB',regex=True).astype(object)
        df = df.replace('^rb','RB',regex=True).astype(object)
        df = df.replace('^wr','WR',regex=True).astype(object)
        df = df.replace('^te','TE',regex=True).astype(object)
        df = df.replace('\+','',regex=True).astype(object)
        df = df.replace('\*','',regex=True).astype(object)
        columns = ['Player','Tm_x','Pos_x','Age_x','G_x','GS_x','Cmp_x','Att_x','Yds_x','IAY',
                   'IAY/PA','CAY','CAY/Cmp','CAY/PA','YAC','YAC/Cmp','Bats','ThAwy',
                   'Spikes','Drops','Drop%','BadTh','Bad%','OnTgt','OnTgt%','Sk',
                   'PktTime','Bltz','Hrry','Prss','Prss%','Scrm','Yds/Scr','Plays',
                   'Yds.1','PassAtt','PassYds','RushAtt','RushYds', 'PassAtt.1', 'PassYds.1']

        df = df[columns]
        df = df.loc[:,~df.columns.duplicated()]
        cols = []
        for column in df.columns:
            if column == 'Tm_x':
                cols.append('Tm')
            elif column == 'Pos_x':
                cols.append('Pos')
            elif column == 'Age_x':
                cols.append('Age')
            elif column == 'G_x':
                cols.append('G')
            elif column == 'GS_x':
                cols.append('GS')
            elif column == 'Cmp_x':
                cols.append('Cmp')
            elif column == 'Att_x':
                cols.append('Att')
            elif column == 'Yds_x':
                cols.append('Yds')
            elif column == 'Yds.1':
                cols.append('RPO_Yds')
            elif column == 'PassAtt':
                cols.append('RPO_PassAtt')
            elif column == 'PassYds':
                cols.append('RPO_PassYds')
            elif column == 'RushAtt':
                cols.append('RPO_RushAtt')
            elif column == 'RushYds':
                cols.append('RPO_RushYds')
            elif column == 'PassAtt.1':
                cols.append('PA_PassAtt')
            elif column == 'PassYds.1':
                cols.append('PA_PassYds')
            else:
                cols.append(column)

        df.columns = cols
        df = df.fillna(0)
        df = df[df['Pos'] != 'k']
        df = df[df['Pos'] != 'p']
        columns = df.columns[3:]
        for col in columns:
            if '%' in col:
                df[col] = df[col].str.rstrip('%').astype('float') / 100.0
            else:
                df[col] = pd.to_numeric(df[col],errors='coerce')
                
        df['Year'] = year
        df['Player'] = df.Player.str.replace('[^a-zA-Z]', '', regex = True)
        # creates unique player ID to make stats for different years easier to see
        playerID = []
        for index, row in df.iterrows():
            playerID.append(row['Player'] + str(row['Year']))
        df['ID'] = playerID
    except:
        
        df1 = html_soup.find('table', {"id" : 'advanced_air_yards'})
        df2 = html_soup.find('table', {"id" : 'advanced_accuracy'})
        df3 = html_soup.find('table', {"id" : 'advanced_pressure'})

        df1 = pd.read_html(str(df1),header=1)[0]
        df2 = pd.read_html(str(df2),header=1)[0]
        df3 = pd.read_html(str(df3),header=1)[0]
    
        data_frames = [df1, df2, df3]
        df = reduce(lambda  left,right: pd.merge(left,right,on=['Player'],
                                                    how='outer'), data_frames)
        df = df[df['Player'] != 'Player']
        df = df[df['Player'].notna()]
        df = df.replace('\/','',regex=True).astype(object)
        df = df.replace('^qb','QB',regex=True).astype(object)
        df = df.replace('^rb','RB',regex=True).astype(object)
        df = df.replace('^wr','WR',regex=True).astype(object)
        df = df.replace('^te','TE',regex=True).astype(object)
        df = df.replace('\+','',regex=True).astype(object)
        df = df.replace('\*','',regex=True).astype(object)
        columns = ['Player','Tm_x','Pos_x','Age_x','G_x','GS_x','Cmp_x','Att_x','Yds_x','IAY',
                   'IAY/PA','CAY','CAY/Cmp','CAY/PA','YAC','YAC/Cmp','ThAwy',
                   'Spikes','Drops','Drop%','BadTh','Bad%','Sk',
                   'PktTime','Bltz','Hrry','Prss','Prss%','Scrm','Yds/Scr']

        df = df[columns]
        df = df.loc[:,~df.columns.duplicated()]
        cols = []
        for column in df.columns:
            if column == 'Tm_x':
                cols.append('Tm')
            elif column == 'Pos_x':
                cols.append('Pos')
            elif column == 'Age_x':
                cols.append('Age')
            elif column == 'G_x':
                cols.append('G')
            elif column == 'GS_x':
                cols.append('GS')
            elif column == 'Cmp_x':
                cols.append('Cmp')
            elif column == 'Att_x':
                cols.append('Att')
            elif column == 'Yds_x':
                cols.append('Yds')
            else:
                cols.append(column)

        df.columns = cols
        df = df.fillna(0)
        df = df[df['Pos'] != 'k']
        df = df[df['Pos'] != 'p']
        columns = df.columns[3:]
        for col in columns:
            if '%' in col:
                df[col] = df[col].str.rstrip('%').astype('float') / 100.0
            else:
                df[col] = pd.to_numeric(df[col],errors='coerce')
                
        df['Year'] = year
        df['Player'] = df.Player.str.replace('[^a-zA-Z]', '', regex = True)
        # creates unique player ID to make stats for different years easier to see
        playerID = []
        for index, row in df.iterrows():
            playerID.append(row['Player'] + str(row['Year']))
        df['ID'] = playerID
    
    browser.quit()
    
    return df

In [ ]:
def scrapeAdvancedRush(year):
    from splinter import Browser
    from bs4 import BeautifulSoup as soup
    from webdriver_manager.chrome import ChromeDriverManager
    import pandas as pd
    # Set up Splinter
    executable_path = {'executable_path': ChromeDriverManager().install()}
    browser = Browser('chrome', **executable_path, headless=True)
    # Visit the AdvancedStats to Scrape site
    url = f'https://www.pro-football-reference.com/years/{year}/rushing_advanced.htm'
    browser.visit(url)
    # Parse the HTML
    html = browser.html
    html_soup = soup(html, 'html.parser')
    
    df = html_soup.find('table', {"id" : 'advanced_rushing'})
    
    df = pd.read_html(str(df),header=1)[0]
    
    df = df[df['Player'] != 'Player']
    df = df[df['Player'].notna()]
    df = df.replace('\/','',regex=True).astype(object)
    df = df.replace('^qb','QB',regex=True).astype(object)
    df = df.replace('^rb','RB',regex=True).astype(object)
    df = df.replace('^wr','WR',regex=True).astype(object)
    df = df.replace('^te','TE',regex=True).astype(object)
    df = df.replace('\+','',regex=True).astype(object)
    df = df.replace('\*','',regex=True).astype(object)
    columns = ['Player','Tm','Pos','Age','G','GS','Att','Yds','1D',
               'YBC','YBC/Att','YAC','YAC/Att','BrkTkl','Att/Br']

    df = df[columns]

    df = df.fillna(0)
    df = df[df['Pos'] != 'k']
    df = df[df['Pos'] != 'p']
    columns = df.columns[3:]
    for col in columns:
        df[col] = pd.to_numeric(df[col],errors='coerce')
    
    df['Year'] = year
    df['Player'] = df.Player.str.replace('[^a-zA-Z]', '', regex = True)
    # creates unique player ID to make stats for different years easier to see
    playerID = []
    for index, row in df.iterrows():
        playerID.append(row['Player'] + str(row['Year']))
    df['ID'] = playerID
    
    browser.quit()
    
    return df

In [ ]:
def scrapeAdvancedRec(year):
    from splinter import Browser
    from bs4 import BeautifulSoup as soup
    from webdriver_manager.chrome import ChromeDriverManager
    import pandas as pd
    # Set up Splinter
    executable_path = {'executable_path': ChromeDriverManager().install()}
    browser = Browser('chrome', **executable_path, headless=True)
    # Visit the AdvancedStats to Scrape site
    url = f'https://www.pro-football-reference.com/years/{year}/receiving_advanced.htm'
    browser.visit(url)
    # Parse the HTML
    html = browser.html
    html_soup = soup(html, 'html.parser')
    
    df = html_soup.find('table', {"id" : 'advanced_receiving'})
    
    df = pd.read_html(str(df),header=0)[0]
    
    df = df[df['Player'] != 'Player']
    df = df[df['Player'].notna()]
    df = df.replace('\/','',regex=True).astype(object)
    df = df.replace('^qb','QB',regex=True).astype(object)
    df = df.replace('^rb','RB',regex=True).astype(object)
    df = df.replace('^wr','WR',regex=True).astype(object)
    df = df.replace('^te','TE',regex=True).astype(object)
    df = df.replace('\+','',regex=True).astype(object)
    df = df.replace('\*','',regex=True).astype(object)
    columns = ['Player','Tm','Pos','Age','G','GS','Tgt','Rec','Yds','TD','1D','YBC','YBC/R',
               'YAC','YAC/R','ADOT','BrkTkl','Rec/Br','Drop','Drop%','Int','Rat']

    df = df[columns]

    df = df.fillna(0)
    df = df[df['Pos'] != 'k']
    df = df[df['Pos'] != 'p']
    columns = df.columns[3:]
    for col in columns:
        if '%' in col:
            df[col] = df[col].str.rstrip('%').astype('float') / 100.0
        else:
            df[col] = pd.to_numeric(df[col],errors='coerce')
    
    df['Year'] = year
    df['Player'] = df.Player.str.replace('[^a-zA-Z]', '', regex = True)
    # creates unique player ID to make stats for different years easier to see
    playerID = []
    for index, row in df.iterrows():
        playerID.append(row['Player'] + str(row['Year']))
    df['ID'] = playerID
    
    browser.quit()
    
    return df

In [ ]:
def cleanffball(year): # add name to inputs if wanting to create csv
    import pandas as pd
    url = f'https://www.pro-football-reference.com/years/{year}/fantasy.htm'
    df = pd.read_html(url,header=1)[0]
    df = df.drop(['Rk','Tm','VBD','FantPt','DKPt','FDPt'],axis=1)
    df = df[df['FantPos'].notna()]
    df = df[df['FantPos'] != 'FantPos']
    df['Y/A'] = df['Y/A'].fillna(0)
    df['Y/R'] = df['Y/R'].fillna(0)
    df['2PM'] = df['2PM'].fillna(0)
    df['2PP'] = df['2PP'].fillna(0)
    df = df.replace('\+','',regex=True).astype(object)
    df = df.replace('\*','',regex=True).astype(object)
    df['Player'] = df['Player'].str.rstrip()
    columns = df.columns[2:]
    for col in columns:
        df[col] = pd.to_numeric(df[col],errors='coerce')
    df['PPR_OvRank'] = df['PPR'].rank(method='first',ascending=False)
    df = df.drop(columns=['OvRank'],axis=1)
    df = df.dropna()
    df['Year'] = year
    df['PPG'] = df['PPR'] / df['G']
    df['PPG'] = df['PPG'].round(decimals = 3)
    # edits name for uniformity year over year
    df['Player'] = df.Player.str.replace('[^a-zA-Z]', '', regex = True)
    # creates unique player ID to make stats for different years easier to see
    playerID = []
    for index, row in df.iterrows():
        playerID.append(row['Player'] + str(row['Year']))
    df['ID'] = playerID
    return df

In [ ]:
def createDB(startYear,endYear,function):
    database = pd.DataFrame()
    years = []
    # iterates through years specified
    for year in range(startYear, endYear + 1):
        years.append(year)
    for year in years:
        database = database.append(function(year))
    return database

In [ ]:
def cleanDB(database):
    cols = []
    for column in database.columns:
        if column == 'FantPos':
            cols.append('Pos')
        elif column == 'Yds':
            cols.append('PassYds')
        elif column == 'Yds.1':
            cols.append('RushYds')
        elif column == 'Yds.2':
            cols.append('RecYds')
        elif column == 'Att':
            cols.append('PassAtt')
        elif column == 'Att.1':
            cols.append('RushAtt')
        elif column == 'TD':
            cols.append('PassTD')
        elif column == 'TD.1':
            cols.append('RushTD')
        elif column == 'TD.2':
            cols.append('RecTD')
        elif column == 'TD.3':
            cols.append('TotTD')
        else:
            cols.append(column)

    database.columns = cols
    
    listOfColumns = ['Cmp','PassAtt','PassYds','PassTD','Int','RushAtt','RushYds',
                     'Y/A','RushTD','Tgt','Rec','RecYds','Y/R','RecTD','Fmb',
                     'FL','TotTD','2PM','2PP','PPR','PosRank','PPR_OvRank','Year',
                     'PPG']
    
    # Creates columns for stats per game
    for column in listOfColumns:
        if column == 'Y/A':
            newColumn = 'RushYds/Att'
            database[newColumn] = database[column]
        elif column == 'Y/R':
            newColumn = 'Yds/Rec'
            database[newColumn] = database[column]
        elif column == 'PPR':
            newColumn = 'FPts'
            database[newColumn] = database[column]
        elif column == 'PosRank':
            newColumn = 'PosRk'
            database[newColumn] = database[column]
        elif column == 'PPR_OvRank':
            newColumn = 'OvRank'
            database[newColumn] = database[column]
        elif column == 'Year':
            newColumn = 'Yr'
            database[newColumn] = database[column]
        elif column == 'PPG':
            newColumn = 'PPR/G'
            database[newColumn] = database[column]
        else:
            newColumn = column + '/G'
            database[newColumn] = (pd.to_numeric(database[column]) / pd.to_numeric(database['G']))
            database[newColumn] = database[newColumn].round(decimals = 3)
            
    database = database.drop(columns = listOfColumns)
    database.loc[(database['Pos'] == 'QB') & ((database['GS'] / database['G']) >= .75) & (database['GS'] >= 5), 'Starter'] = 1
    database.loc[(database['Pos'] == 'QB') & ((database['GS'] / database['G']) < .75) & (database['GS'] < 5), 'Starter'] = 0 
    database['Starter'] = database['Starter'].fillna(0)
    
    database = database.sort_values(by = ['Player','Yr'], ascending = [True,False])
    nextYearPPG = []
    nextYearStart = []
    lastPlayer = 'NaN'
    lastPlayerPPG = 'NaN'
    lastPlayerStart = 'NaN'

    for index, row in database.iterrows():
            player = row['Player']
            if lastPlayer == player:
                nextYearPPG.append(lastPlayerPPG)
                nextYearStart.append(lastPlayerStart)
            else:
                nextYearPPG.append('NaN')
                nextYearStart.append('NaN')
            lastPlayer = row['Player']
            lastPlayerPPG = row['PPR/G']
            lastPlayerStart = row['Starter']
    
    database['Next_Yr_PPG'] = nextYearPPG
    database['Next_Yr_PPG'] = pd.to_numeric(database['Next_Yr_PPG'],errors='coerce')
    database['Next_Yr_Starter'] = nextYearStart
    database['Next_Yr_Starter'] = pd.to_numeric(database['Next_Yr_Starter'],errors='coerce',downcast='integer')
    database = database.reset_index(drop = True)
    return database

In [ ]:
import pandas as pd
pd.set_option('display.max_rows', None)
pd.set_option('display.max_columns', None)

In [ ]:
adv_pass = createDB(2019,2021,scrapeAdvancedPass)
adv_rush = createDB(2019,2021,scrapeAdvancedRush)
adv_rec = createDB(2019,2021,scrapeAdvancedRec)

In [ ]:
db = cleanDB(createDB(2012, 2021,cleanffball))

In [ ]:
db_pos = db[['ID','Pos']]

In [ ]:
adv_pass = adv_pass.merge(db_pos,how='inner',on='ID')
adv_pass.drop(columns='Pos_x',inplace=True)
cols = ['Player', 'Tm', 'Pos_y', 'Age', 'G', 'GS', 'Cmp', 'Att', 'Yds', 'IAY',
       'IAY/PA', 'CAY', 'CAY/Cmp', 'CAY/PA', 'YAC', 'YAC/Cmp', 'Bats', 'ThAwy',
       'Spikes', 'Drops', 'Drop%', 'BadTh', 'Bad%', 'OnTgt', 'OnTgt%', 'Sk',
       'PktTime', 'Bltz', 'Hrry', 'Prss', 'Prss%', 'Scrm', 'Yds/Scr', 'Plays',
       'RPO_Yds', 'RPO_PassAtt', 'RPO_PassYds', 'RPO_RushAtt', 'RPO_RushYds',
       'PA_PassAtt', 'PA_PassYds', 'Year', 'ID']
adv_pass = adv_pass[cols]
adv_pass.rename(columns={'Pos_y':'Pos'}, inplace=True)

In [ ]:
adv_rush = adv_rush.merge(db_pos,how='inner',on='ID')
adv_rush.drop(columns='Pos_x',inplace=True)
cols = ['Player', 'Tm', 'Pos_y', 'Age', 'G', 'GS', 'Att', 'Yds', '1D', 'YBC',
       'YBC/Att', 'YAC', 'YAC/Att', 'BrkTkl', 'Att/Br', 'Year', 'ID']
adv_rush = adv_rush[cols]
adv_rush.rename(columns={'Pos_y':'Pos'}, inplace=True)

In [ ]:
adv_rec = adv_rec.merge(db_pos,how='inner',on='ID')
adv_rec.drop(columns='Pos_x',inplace=True)
cols = ['Player', 'Tm', 'Pos_y', 'Age', 'G', 'GS', 'Tgt', 'Rec', 'Yds', 'TD',
       '1D', 'YBC', 'YBC/R', 'YAC', 'YAC/R', 'ADOT', 'BrkTkl', 'Rec/Br',
       'Drop', 'Drop%', 'Int', 'Rat', 'Year', 'ID']
adv_rec = adv_rec[cols]
adv_rec.rename(columns={'Pos_y':'Pos'}, inplace=True)

In [ ]:
db

In [ ]:
adv_pass

In [ ]:
future = db[['ID','Next_Yr_PPG','Next_Yr_Starter']]
future

In [ ]:
len(adv_pass)

In [ ]:
adv_pass2 = adv_pass.merge(future,how='inner',on='ID')
adv_pass2

In [ ]:
len(adv_pass2)

In [ ]:
len(adv_rush)

In [ ]:
adv_rush2 = adv_rush.merge(future,how='inner',on='ID')
adv_rush2

In [ ]:
len(adv_rush2)

In [ ]:
len(adv_rec)

In [ ]:
adv_rec2 = adv_rec.merge(future,how='inner',on='ID')
adv_rec2

In [ ]:
len(adv_rec2)

In [ ]:
adv_pass2.to_csv('advancedpassing.csv')
adv_rush2.to_csv('advancedrushing.csv')
adv_rec2.to_csv('advancedreceiving.csv')
db.to_csv('fantasy.csv')